In [ ]:
# Let's build our project

# Here we are building a mulitple input mixed data model that takes in the 3 inputs seperately. RBI, chaneel matrices remain 
# seperate but now instead of giving Spectrum Occupancy matrix as input we give only NSA and instead of Src Des matrix we 
# just give Src and Des node id. The NSA and Src_Des input are given in two different branches, in the next model they will 
# be given together on one branch. The output power are indices rather matrices. So its an Nx1 vector.

import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

print(tf.config.list_physical_devices())

# number of iterations the matlab code ran or the number of times the network is simulated
itr = 220606
# Number of iterations the test data generation matlab code ran or the number of times the network is simulated
itr_test = 65
# number of nodes in the network
N = 30
# We scale the non zero values of output power so that there is some sizable error if the model outputs a zero matrix
scaling_parameter = 1

# Reading the RBI data
df = pd.read_csv("Data_rbi_vectors.csv")
#print(df)
df.shape

In [ ]:
# coverting it into numpy array
X_1 = df.to_numpy()
#print(tem)
X_1.shape

#Just for the 1332 dataset as the last array will be nans
#tem = tem[:-1]

In [ ]:
# a=list(range(0, itr))
# X_1 = np.zeros((itr,N))
# for i in a:
#     X_1[i:] = tem[i:]
#X_1

In [ ]:
# Reading the channel data
df1 = pd.read_csv("Data_channel_matrices/Data_channel_matrices.csv")
#print(df1)
df1.shape

In [ ]:
# coverting it into numpy array
X_2 = df1.to_numpy()

In [ ]:
X_2 = X_2.reshape((78092,N,N))
#print(X_2)
X_2 = np.nan_to_num(X_2)
#X_2

In [ ]:
# Reading the channel data
df1a = pd.read_csv("Data_channel_matrices/Data_channel_matrices1.csv")
#print(df1a)
df1a.shape

In [ ]:
# coverting it into numpy array
X_2_a = df1a.to_numpy()

In [ ]:
X_2_a = X_2_a.reshape((89998,N,N))
#print(X_2_a)
X_2_a = np.nan_to_num(X_2_a)
#X_2

In [ ]:
# Reading the channel data
df1b = pd.read_csv("Data_channel_matrices/Data_channel_matrices2.csv")
#print(df1b)
df1b.shape

In [ ]:
# converting it into numpy array
X_2_b = df1b.to_numpy()

In [ ]:
X_2_b = X_2_b.reshape((52516,N,N))
#print(X_2_b)
X_2_b = np.nan_to_num(X_2_b)

In [ ]:
# Stacking X_2 above X_2_a
X_2_new = np.concatenate((X_2, X_2_a, X_2_b), axis=0)
X_2_new.shape
if (X_2_new[78094,:,:] == X_2_a[2,:,:]).all():
    print("true")
if (X_2_new[168092,:,:] == X_2_b[2, :, :]).all():
    print("true again for the Data_channel_matrices2 ")
    

In [ ]:
X_2 = X_2_new

In [ ]:
# Reading the Source_Destination_info data
df2 = pd.read_csv("Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2)
df2.shape

In [ ]:
# coverting it into numpy array
X_3 = df2.to_numpy()
#print(X_3)
X_3.shape

In [ ]:
X_3 = X_3.astype(np.float64)
print(X_3.dtype)
#print(X_3)

In [ ]:
# Reading the Spectrum_occupancy data, i.e, NSA
df3 = pd.read_csv("Data_spectrum_matrix.csv")
#print(df3)
df3.shape

In [ ]:
# coverting it into numpy array
X_4 = df3.to_numpy()
#print(X_4)
X_4.shape

# Its already in float64 so no need to convert

In [ ]:
# Reading the output power data
df4 = pd.read_csv("Data_output1.csv")
#print(df4)
df4.shape

In [ ]:
# coverting it into numpy array
X_5 = df4.to_numpy()
#print(X_5)
print(X_5.shape)
X_5 = np.nan_to_num(X_5)

In [ ]:
# This Cell converts each of the (1,30) output of y_data to (1,6)

a=list(range(0, itr))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_new = np.zeros((itr,6))
for i in a:
    temp = X_5[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_new[i,5] = temp_non_zero[-1]
        continue
    X_5_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_new.shape

In [ ]:
# Note: for 33569 itr data  the 150th and 7128 paths have length > 6 , ie, [ 6. 23. 27.  3. 15. 17. 18.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.] and [30.  7.  8. 16.  3. 25. 12. 21.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
#  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.]

In [ ]:
y_data = X_5_new.astype(np.float64)
print(y_data.dtype)
#print(y_data_temp)
y_data.shape

# No need to unravel it as its already a vector

In [ ]:
X_1.dtype, X_2.dtype, X_3.dtype, X_4.dtype, y_data.dtype

In [ ]:
X_1.shape, X_2.shape, X_3.shape, X_4.shape, y_data.shape

In [ ]:
# scaling parameter for y_data so model gets greater penalty for error and also since output is sparse

y_data = y_data*scaling_parameter

In [ ]:
# converting y_data to tensor

y_data = tf.constant(y_data)
print(y_data.shape)
#y_data = tf.reshape(y_data, [itr,1,1,N*N])
#print(y_data.shape)

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers.core import Activation
from tensorflow.python.keras.layers.core import Dense
from tensorflow.python.keras.layers import Dropout
from tensorflow.python.keras.layers import Flatten
from tensorflow.python.keras.layers import Input
from tensorflow.python.keras.models import Model
from tensorflow.python.keras.layers import concatenate
#from tensorflow.python.keras.optimizers import adam

In [ ]:
# Creating branches of the model

# Activations
a1 = 'relu'
a2 = None

# Branch 1, takes RBI vectors as input and processws it

rbi_input = Input(shape=(N,))
x1 = tf.keras.layers.BatchNormalization()(rbi_input)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dropout(0.3)(x1)
x1 = Dense(50, activation=a1)(x1)
#x1 = Dense(1000, activation=a2)(x1)
#x1 = Dense(1000, activation=a2)(x1)
out_1 = Dense(N, activation=a1)(x1)

# Branch 2, takes channel matrix as input and processes it

chan_input = Input(shape=(N, N))
x2 = tf.keras.layers.BatchNormalization()(chan_input)
#x2 = Dropout(0.3)(x2)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dropout(0.3)(x2)
#x2 = Flatten()(chan_input)
x2 = Dense(80, activation=a1)(x2)
#x2 = Dense(1000, activation =a2)(x2)
#x2 = Dense(1000, activation =a2)(x2)
x2 = Flatten()(x2)
out_2 = Dense(N*N, activation=a1)(x2)

# Branch 3, takes Source Destination info and processes it

S_D_input = Input(shape=(2, ))
#x3 = Flatten()(S_D_input)
x3 = tf.keras.layers.BatchNormalization()(S_D_input)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dropout(0.3)(x3)
x3 = Dense(10, activation=a1)(x3)
#x3 = Dropout(0.2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Dense(1000, activation =a2)(x3)
#x3 = Flatten()(x3)
out_3 = Dense(10, activation=a1)(x3)

# Branch 4, takes Spectrum occupancy info and processes it

Spec_input = Input(shape=(1, ))
#x4 = Flatten()(Spec_input)
x4 = tf.keras.layers.BatchNormalization()(Spec_input)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dropout(0.3)(x4)
x4 = Dense(20, activation=a1)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Dense(1000, activation =a2)(x4)
#x4 = Flatten()(x4)
out_4 = Dense(10, activation=a1)(x4)

# Now we concatenate all the branches 

concatenated = concatenate([out_1, out_2, out_3, out_4]) ## concatenate the four branches
#x5 = Dropout(0.1)(cocatenated)
x5 = Dense(100, activation=a1)(concatenated)
#x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
#x5 = Dropout(0.3)(x5)
x5 = Dense(100, activation=a1)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dense(2000, activation=a2)(x5)
#x5 = Dropout(0.2)(x5)
out = Dense(6, activation=a1)(x5)
model = Model([rbi_input, chan_input, S_D_input, Spec_input], out, name='Model_3')

# Set optimizer

#opt = Adam(lr=0.01, decay=1e-6)
#sgd = SGD(lr=0.0, momentum=0.9)

# Compile the model

#model.compile(loss='mse',
#              optimizer = 'adam',
#              metrics=['accuracy'])

model.compile(loss=tf.keras.losses.mse, # mae is short for mean absolute error
              optimizer=tf.keras.optimizers.Adam(learning_rate = 0.001), # SGD is short for stocastic gradient descent
              metrics=["accuracy"])

#model.compile(loss=tf.keras.losses.mse,
#              optimizer=tf.keras.optimizers.Adam(),
#              metrics=["accuracy"])

# Fit the model

history = model.fit([X_1, X_2, X_3, X_4], y_data, batch_size=64, epochs=1850, validation_split = 0.03)
#next try batch size of 32
#The s2nd dropout gave less accuracy than just the one dropout


3344/3344 [==============================] - 33s 10ms/step - loss: 5.7673 - accuracy: 0.8999 - val_loss: 63.2321 - val_accuracy: 0.5957
Epoch 109/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.6611 - accuracy: 0.9018 - val_loss: 64.3727 - val_accuracy: 0.5841
Epoch 110/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.6081 - accuracy: 0.9018 - val_loss: 63.7042 - val_accuracy: 0.5897
Epoch 111/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.5993 - accuracy: 0.9008 - val_loss: 64.3691 - val_accuracy: 0.5859
Epoch 112/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.6125 - accuracy: 0.9012 - val_loss: 64.3971 - val_accuracy: 0.5928
Epoch 113/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.5241 - accuracy: 0.9019 - val_loss: 64.7972 - val_accuracy: 0.5786
Epoch 114/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 5.5093 - accuracy: 0.9017 -

3344/3344 [==============================] - 33s 10ms/step - loss: 3.5150 - accuracy: 0.9212 - val_loss: 66.0015 - val_accuracy: 0.5842
Epoch 216/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.4624 - accuracy: 0.9213 - val_loss: 65.4060 - val_accuracy: 0.5789
Epoch 217/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.4410 - accuracy: 0.9219 - val_loss: 64.9927 - val_accuracy: 0.5829
Epoch 218/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.4827 - accuracy: 0.9213 - val_loss: 65.7773 - val_accuracy: 0.5818
Epoch 219/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.4528 - accuracy: 0.9220 - val_loss: 64.4355 - val_accuracy: 0.5900
Epoch 220/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.4309 - accuracy: 0.9212 - val_loss: 64.7473 - val_accuracy: 0.5953
Epoch 221/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 3.3913 - accuracy: 0.9224 -

3344/3344 [==============================] - 33s 10ms/step - loss: 2.8146 - accuracy: 0.9295 - val_loss: 65.6336 - val_accuracy: 0.5850
Epoch 324/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.7969 - accuracy: 0.9294 - val_loss: 65.9236 - val_accuracy: 0.5910
Epoch 325/1850
3344/3344 [==============================] - 32s 10ms/step - loss: 2.8163 - accuracy: 0.9282 - val_loss: 64.9904 - val_accuracy: 0.5844
Epoch 326/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.8165 - accuracy: 0.9287 - val_loss: 63.7896 - val_accuracy: 0.5874
Epoch 327/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.8162 - accuracy: 0.9288 - val_loss: 63.9510 - val_accuracy: 0.5886
Epoch 328/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.8042 - accuracy: 0.9293 - val_loss: 64.9061 - val_accuracy: 0.5886
Epoch 329/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.7760 - accuracy: 0.9308 -

3344/3344 [==============================] - 33s 10ms/step - loss: 2.4827 - accuracy: 0.9326 - val_loss: 64.3220 - val_accuracy: 0.5907
Epoch 432/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.4973 - accuracy: 0.9320 - val_loss: 64.2261 - val_accuracy: 0.5795
Epoch 433/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.5151 - accuracy: 0.9318 - val_loss: 62.8917 - val_accuracy: 0.5978
Epoch 434/1850
3344/3344 [==============================] - 32s 10ms/step - loss: 2.5114 - accuracy: 0.9312 - val_loss: 65.7260 - val_accuracy: 0.5786
Epoch 435/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.5009 - accuracy: 0.9325 - val_loss: 65.4761 - val_accuracy: 0.5841
Epoch 436/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.4894 - accuracy: 0.9331 - val_loss: 64.8296 - val_accuracy: 0.5805
Epoch 437/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.4992 - accuracy: 0.9321 -

3344/3344 [==============================] - 32s 10ms/step - loss: 2.2977 - accuracy: 0.9361 - val_loss: 64.8832 - val_accuracy: 0.5901
Epoch 540/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.3108 - accuracy: 0.9344 - val_loss: 64.6962 - val_accuracy: 0.5850
Epoch 541/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.3600 - accuracy: 0.9348 - val_loss: 65.6585 - val_accuracy: 0.5895
Epoch 542/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.3346 - accuracy: 0.9341 - val_loss: 64.1751 - val_accuracy: 0.5936
Epoch 543/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.2798 - accuracy: 0.9345 - val_loss: 64.7551 - val_accuracy: 0.5891
Epoch 544/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.3003 - accuracy: 0.9365 - val_loss: 64.4165 - val_accuracy: 0.5759
Epoch 545/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 2.3266 - accuracy: 0.9353 -

...

3344/3344 [==============================] - 33s 10ms/step - loss: 1.8253 - accuracy: 0.9410 - val_loss: 64.8605 - val_accuracy: 0.5933
Epoch 1452/1850
3344/3344 [==============================] - 32s 10ms/step - loss: 1.7910 - accuracy: 0.9418 - val_loss: 67.7103 - val_accuracy: 0.5666
Epoch 1453/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8523 - accuracy: 0.9405 - val_loss: 64.4252 - val_accuracy: 0.5953
Epoch 1454/1850
3344/3344 [==============================] - 32s 10ms/step - loss: 1.8245 - accuracy: 0.9411 - val_loss: 65.3067 - val_accuracy: 0.5842
Epoch 1455/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8017 - accuracy: 0.9414 - val_loss: 64.5365 - val_accuracy: 0.5962
Epoch 1456/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8508 - accuracy: 0.9399 - val_loss: 65.7207 - val_accuracy: 0.5820
Epoch 1457/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8045 - accuracy: 0.

3344/3344 [==============================] - 33s 10ms/step - loss: 1.8185 - accuracy: 0.9409 - val_loss: 65.6351 - val_accuracy: 0.5925
Epoch 1558/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8067 - accuracy: 0.9409 - val_loss: 65.9765 - val_accuracy: 0.5818
Epoch 1559/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8164 - accuracy: 0.9410 - val_loss: 65.9128 - val_accuracy: 0.5892
Epoch 1560/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7722 - accuracy: 0.9419 - val_loss: 65.2867 - val_accuracy: 0.5863
Epoch 1561/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7676 - accuracy: 0.9413 - val_loss: 66.4917 - val_accuracy: 0.5777
Epoch 1562/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8041 - accuracy: 0.9417 - val_loss: 66.6700 - val_accuracy: 0.5845
Epoch 1563/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8445 - accuracy: 0.

3344/3344 [==============================] - 33s 10ms/step - loss: 1.7817 - accuracy: 0.9416 - val_loss: 64.4531 - val_accuracy: 0.5922
Epoch 1664/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7864 - accuracy: 0.9414 - val_loss: 65.8812 - val_accuracy: 0.5900
Epoch 1665/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7912 - accuracy: 0.9409 - val_loss: 66.6744 - val_accuracy: 0.5888
Epoch 1666/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7727 - accuracy: 0.9414 - val_loss: 65.4766 - val_accuracy: 0.5850
Epoch 1667/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7404 - accuracy: 0.9415 - val_loss: 64.8883 - val_accuracy: 0.5820
Epoch 1668/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8174 - accuracy: 0.9403 - val_loss: 65.8128 - val_accuracy: 0.5832
Epoch 1669/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7037 - accuracy: 0.

3344/3344 [==============================] - 33s 10ms/step - loss: 1.8233 - accuracy: 0.9413 - val_loss: 64.7139 - val_accuracy: 0.5874
Epoch 1718/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8387 - accuracy: 0.9408 - val_loss: 65.4685 - val_accuracy: 0.5888
Epoch 1719/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7545 - accuracy: 0.9417 - val_loss: 65.1871 - val_accuracy: 0.5916
Epoch 1720/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7461 - accuracy: 0.9420 - val_loss: 65.9843 - val_accuracy: 0.5841
Epoch 1721/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8018 - accuracy: 0.9412 - val_loss: 65.1289 - val_accuracy: 0.5789
Epoch 1722/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7915 - accuracy: 0.9407 - val_loss: 66.8578 - val_accuracy: 0.5823
Epoch 1723/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7563 - accuracy: 0.

Epoch 1771/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.8125 - accuracy: 0.9401 - val_loss: 66.3226 - val_accuracy: 0.5880
Epoch 1772/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7450 - accuracy: 0.9414 - val_loss: 66.1836 - val_accuracy: 0.5786
Epoch 1773/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7915 - accuracy: 0.9413 - val_loss: 66.1279 - val_accuracy: 0.5913
Epoch 1774/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7750 - accuracy: 0.9413 - val_loss: 65.2684 - val_accuracy: 0.5876
Epoch 1775/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7581 - accuracy: 0.9422 - val_loss: 67.0784 - val_accuracy: 0.5811
Epoch 1776/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.7682 - accuracy: 0.9417 - val_loss: 65.5389 - val_accuracy: 0.5783
Epoch 1777/1850
3344/3344 [==============================] - 33s 10ms/step - loss: 1.772

In [ ]:
model.summary()

In [ ]:
# Now we read the test data to evaluate our model


# Reading the Test RBI data
df_test = pd.read_csv("Test_Data_rbi_vectors.csv")
#print(df_test)
df_test.shape

In [ ]:
# coverting it into numpy array
tem_test = df_test.to_numpy()
#print(tem_test)
tem_test.shape
X_1_test = tem_test

In [ ]:
# Reading the Test channel data
df1_test = pd.read_csv("Test_Data_channel_matrices.csv")
#print(df1_test)
df1_test.shape

In [ ]:
# coverting it into numpy array
X_2_test = df1_test.to_numpy()
X_2_test.shape

In [ ]:
X_2_test = X_2_test.reshape((itr_test,N,N))
#print(X_2_test)
X_2_test = np.nan_to_num(X_2_test)
#X_2_test

In [ ]:
# Reading the Test Source_Destination_info data

df2_test = pd.read_csv("Test_Data_Source_Destination_info_matrix(only Src Des).csv")
#print(df2_test)
df2_test.shape

In [ ]:
# coverting it into numpy array
X_3_test = df2_test.to_numpy()
#print(X_3_test)
X_3_test.shape

In [ ]:
X_3_test = X_3_test.astype(np.float64)
print(X_3_test.dtype)
#print(X_3_test)

In [ ]:
# Reading the Test Spectrum_occupancy data, i.e, NSA

df3_test = pd.read_csv("Test_Data_spectrum_matrix.csv")
#print(df3_test)
df3_test.shape

In [ ]:
# coverting it into numpy array
X_4_test = df3_test.to_numpy()
#print(X_4_test)
X_4_test.shape

In [ ]:
X_4_test = X_4_test.astype(np.float64)
print(X_4_test.dtype)
#print(X_4_test)

In [ ]:
# Reading the Test output power data

df4_test = pd.read_csv("Test_Data_output1.csv")
#print(df4_test)
df4_test.shape

In [ ]:
# coverting it into numpy array
X_5_test = df4_test.to_numpy()
#print(X_5_test)
print(X_5_test.shape)
X_5_test = np.nan_to_num(X_5_test)

In [ ]:
# This Cell converts each of the (1,30) y_data_test output to (1,6)

a=list(range(0, itr_test))
max1 = 0
count = 0
maxlen = 0
max_path = np.zeros(30)
max_path_ind = 0
X_5_test_new = np.zeros((itr_test,6))
for i in a:
    temp = X_5_test[i]
    maxlen = len(temp[np.nonzero(temp)])
    if maxlen > 6:
        X_5_test_new[i,:] = temp[0:6]
        temp_non_zero = temp[np.nonzero(temp)]
        X_5_test_new[i,5] = temp_non_zero[-1]
        continue
    X_5_test_new[i,:] = temp[0:6]
    #print("original")
    #print(X_5[i])
X_5_test_new.shape

In [ ]:
y_data_test = X_5_test_new.astype(np.float64)
print(y_data_test.dtype)
#print(y_data_test)
y_data_test.shape

In [ ]:
X_1_test.dtype, X_2_test.dtype, X_3_test.dtype, X_4_test.dtype, y_data_test.dtype

In [ ]:
X_1_test.shape, X_2_test.shape, X_3_test.shape, X_4_test.shape, y_data_test.shape

In [ ]:
X_data_test = [X_1_test, X_2_test, X_3_test, X_4_test]

In [ ]:
# converting y_data_test to tensor

y_data_test = tf.constant(y_data_test)
print(y_data_test.shape)
#y_data_test = tf.reshape(y_data_test, [itr_test,1,1,N*N])
#print(y_data_test.shape)
y_data_test

In [ ]:
# We scale the non zero values of Test output power so that there is some sizable error if the model outputs a zero matrix
#scaling_parameter = 100

y_data_test = y_data_test*scaling_parameter
y_data_test

In [ ]:
# Now we make predictions using our model
import time
t = time.perf_counter()
y_pred = model.predict(X_data_test)
elapsed = time.perf_counter() - t
print(elapsed)
print(y_pred.shape)
print("For 1 prediction")
print(elapsed/itr_test)

In [ ]:
# Evaluate the model on the test

t1 = time.perf_counter()
model.evaluate(X_data_test, y_data_test)
elapsed1 = time.perf_counter() - t1
print(elapsed1)

In [ ]:
x_axis = np.arange(1,(6)+1)
x_axis.shape
ind = 3
y_pred = np.round(y_pred)
print(y_pred[ind]/scaling_parameter)
y_data_test[ind]/scaling_parameter

In [ ]:
a = list(range(0, itr_test))
#for i in a:
#    plt.figure()
#    plt.plot(x_axis,y_data_test[i]/scaling_parameter,'b',x_axis,y_pred[i]/scaling_parameter,'r')
#plt.show()

In [ ]:
# summarize history for accuracy
plt.ylim((0.2, 1))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
#plt.ylim((20, 40))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
model.save('saved_model/model(28-10-2023)')
